# Notebook to webscrape and clean stats.nba.com information
### Information to gathter:
- Player Name
- NBA.com Player ID 
- NBA.com Player Page URL 
- Player Usage Stats from 2013-2020

### Libraries Utalized for webscraping
- Selenium

### Why Selenium instead BeautifulSoup?
The reason Selenium was used was because stats.nba.com site was created with a dynamic Javascript component which means that the sourcecode which BeautifulSoup scrapes from did not include the information needed but called into it via Javascript. This was the work around to get the additional information. 

In [1]:
# import libraries
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd

## 1. Gathering all names available on stats.nba.com

Using Selenium on the stats.nba.com player index, scrape all player names and player pages URLs including all historial players since the current view only shows active players and some atheles might have dropped out or came in during the current season. 


In [2]:
# specify the url
urlpage = 'https://stats.nba.com/players/list/?Historic=Y' 
print(urlpage)
# run firefox webdriver from executable path of your choice
driver = webdriver.Firefox()

https://stats.nba.com/players/list/?Historic=Y


In [3]:
# get web page
driver.get(urlpage)
# execute script to scroll down the page
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
# sleep for 30s
time.sleep(30)
# driver.quit()

In [4]:
# find elements by xpath# at the player name level
results = driver.find_elements_by_xpath("//*[@class='stats-player-list players-list']//*[@class='row collapse players-list__section']//*[@class='players-list__names']//*[@class='players-list__name']")
#check number of results
print('Number of results', len(results))

Number of results 4503


In [5]:
# create empty array to store data
data = []
# loop over results to store name and url
for result in results:
    product_name = result.text
    player = result.find_element_by_tag_name('a')
    link = player.get_attribute("href")
    # append dict to array
    data.append({"player" : product_name, "link" : link})

In [7]:
# close driver 
driver.quit()

In [8]:
# save to pandas dataframe
df = pd.DataFrame(data)
df

,player,link
0,"Abdelnaby, Alaa",https://stats.nba.com/player/76001/
1,"Abdul-Aziz, Zaid",https://stats.nba.com/player/76002/
2,"Abdul-Jabbar, Kareem",https://stats.nba.com/player/76003/
3,"Abdul-Rauf, Mahmoud",https://stats.nba.com/player/51/
4,"Abdul-Wahad, Tariq",https://stats.nba.com/player/1505/
...,...,...
4498,"Zizic, Ante",https://stats.nba.com/player/1627790/
4499,"Zoet, Jim",https://stats.nba.com/player/78647/
4500,"Zopf, Bill",https://stats.nba.com/player/78648/
4501,"Zubac, Ivica",https://stats.nba.com/player/1627826/


In [113]:
# save to csv
df.to_csv('stats_nba_links.csv')

In [ ]:
# check length of URL to remove from player page url to get player id 
len('https://stats.nba.com/player/')

In [ ]:
# get player id 
for x in df['stats_id']:
    df['stats_id'] = df['link'].str[29:-1]

In [13]:
df

,player,link,stats_id
0,"Abdelnaby, Alaa",https://stats.nba.com/player/76001/,76001
1,"Abdul-Aziz, Zaid",https://stats.nba.com/player/76002/,76002
2,"Abdul-Jabbar, Kareem",https://stats.nba.com/player/76003/,76003
3,"Abdul-Rauf, Mahmoud",https://stats.nba.com/player/51/,51
4,"Abdul-Wahad, Tariq",https://stats.nba.com/player/1505/,1505
...,...,...,...
4498,"Zizic, Ante",https://stats.nba.com/player/1627790/,1627790
4499,"Zoet, Jim",https://stats.nba.com/player/78647/,78647
4500,"Zopf, Bill",https://stats.nba.com/player/78648/,78648
4501,"Zubac, Ivica",https://stats.nba.com/player/1627826/,1627826


# 2. Match names to our current active player list

Because the active player list on basketball-reference.com and stat.nba.com are different. We need to do the following steps:

- Match the name format
- Merge the two csvs on name 

This cuts down from 4500 URLS to 1200. 

In [ ]:
name = df["player"].str.split(" ", n = -1, expand = True)
for x in name[0]:
    name['name'] = name[1] + ' ' + name[0]
    df['player'] = name['name'].str[:-1]
df

In [ ]:
# save update to csv
df.to_csv('stats_nba_links.csv')

In [ ]:
# cull down number of players for the years we want to match
# load active list created from basketball-reference.com
import csv
path = 'Project-3/Resources/active_players.csv'
active_players = pd.read_csv(path)
active_players

In [ ]:
active_stat_nba = pd.merge(df, active_players, left_on='player', right_on='name')
active_stat_nba = active_stat_nba.drop(columns=['Unnamed: 0','player','Lineup_name'])
active_stat_nba

## 3. Get individual player usage by season

Player usage URL format will looke like https://stats.nba.com/player/203507/usage/?Season=2015-16&SeasonType=Regular%20Season. 

We will need to pass the Player ID and the Season to get the last 7 years of usage. The two variable will determind the correct page to parse the data and scrape the data correctly for each user. 

In [14]:
#define the seasons needed in the format for the url
seasons = ['13-14','14-15','15-16','16-17','17-18','18-19','19-20']

In [ ]:
# loop the player usage page per season to get the overall usage stats
all_usage = pd.DataFrame()
player_usage = pd.DataFrame()

for x in active_stat_nba['stats_id']:
    for y in seasons:
        urlpage = 'https://stats.nba.com/player/'+ str(x) + '/usage/?Season=20' + str(y) + '&SeasonType=Regular%20Season'
        driver = webdriver.Firefox()
        driver.get(urlpage)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        time.sleep(5)
        results = driver.find_elements_by_xpath("/html/body/main/div[2]/div/div/div[3]/div/div/div/nba-stat-table[1]/div[2]/div[1]/table/tbody/tr/td")
        usage = []
        for use in results:
            yearlyuse = use.text
            usage.append(yearlyuse)
        player_usage = pd.DataFrame([usage])
        player_usage['stats_id'] = x
        all_usage = all_usage.append(player_usage)
        driver.quit()

In [ ]:
all_usage

In [ ]:
all_usage.to_csv('all_usage.csv')